<a href="https://colab.research.google.com/github/manavisrani07/Graph4Recommendations/blob/main/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub
import shutil

# Download the dataset
path = kagglehub.dataset_download("mustafaanandwala/10000-bollywood-dataset")

# Specify destination path
destination_path = "/content/2/"

# Move the dataset to /content/
shutil.move(path, destination_path)

print("Dataset files moved to:", destination_path)

100%|██████████| 0.99M/0.99M [00:00<00:00, 115MB/s]

Extracting files...
Dataset files moved to: /content/2/


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/final_dataset.csv')
df.head()

,imdb-id,movie_name,year_of_release,runtime,IMDB_rating,no_of_votes,plot_description,director,actors,genre,keywords
0,tt10811166,The Kashmir Files,(2022),170 min,8.3,"548,031","The Kashmir Files' is a story, based on video ...",Vivek Agnihotri,"Vivek Agnihotri| Mithun Chakraborty, A...",Drama,"['Kashmiri Pandit Community', ""Kashmir Files'""..."
1,tt1187043,3 Idiots,(2009),170 min,8.4,"387,020",Two friends are searching for their long lost ...,Rajkumar Hirani,"Rajkumar Hirani| Aamir Khan, Madhavan,...","Comedy, Drama","['long lost companion', 'lost companion', 'lon..."
2,tt0986264,Taare Zameen Par,(2007),165 min,8.3,"188,938",An eight-year-old boy is thought to be a lazy ...,Aamir Khan,"Aamir Khan, Amole Gupte| Darsheel Safa...","Drama, Family","['boy is thought', 'lazy trouble-maker', 'stru..."
3,tt5074352,Dangal,(2016),161 min,8.3,"183,452",Former wrestler Mahavir Singh Phogat and his t...,Nitesh Tiwari,"Nitesh Tiwari| Aamir Khan, Sakshi Tanw...","Action, Biography, Drama, Sport","['Mahavir Singh Phogat', 'wrestler Mahavir Sin..."
4,tt2338151,PK,(2014),153 min,8.1,"180,108",An alien on Earth loses the only device he can...,Rajkumar Hirani,"Rajkumar Hirani| Aamir Khan, Anushka S...","Comedy, Drama, Sci-Fi","['alien on Earth', 'Earth loses', 'Earth', 'ch..."


# Pre-processing

## Feature Engineering

### Adding genre feature to dataset according to imdb id

In [78]:
import pandas as pd
from imdb import IMDb

# Initialize IMDb instance
ia = IMDb()

# Function to fetch genres based on IMDb ID
def get_genres_from_imdb(imdb_id):
    try:
        # Check if IMDb ID is valid
        if not imdb_id.startswith('tt'):
            return 'Unknown Genre'

        # Remove the 'tt' prefix to get the numeric part of the IMDb ID
        numeric_id = imdb_id.replace('tt', '')
        movie = ia.get_movie(numeric_id)
        return ", ".join(movie.get('genres', [])) if movie.get('genres') else 'Unknown Genre'
    except Exception as e:
        print(f"Error fetching genres for IMDb ID {imdb_id}: {e}")
        return 'Unknown Genre'

# Load the divided part CSV file (replace with the correct file path)
input_csv_path = '/content/2/10000-bollywood-dataset'  # Replace with the actual file path of your part file
df = pd.read_csv(input_csv_path)

# Add the genres column using the function
df['genre'] = df['imdb-id'].apply(get_genres_from_imdb)

# Save the processed file with genres
output_csv_path = input_csv_path.replace('.csv', '_with_genres.csv')
df_part.to_csv(output_csv_path, index=False)

In [4]:
# Get column information
print(df.info())

# Check for missing values
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   imdb-id           9999 non-null   object
 1   movie_name        9999 non-null   object
 2   year_of_release   9799 non-null   object
 3   runtime           9999 non-null   object
 4   IMDB_rating       9999 non-null   object
 5   no_of_votes       9999 non-null   object
 6   plot_description  9999 non-null   object
 7   director          9999 non-null   object
 8   actors            9534 non-null   object
 9   genre             9999 non-null   object
 10  keywords          9999 non-null   object
dtypes: object(11)
memory usage: 859.4+ KB
None
imdb-id               0
movie_name            0
year_of_release     200
runtime               0
IMDB_rating           0
no_of_votes           0
plot_description      0
director              0
actors              465
genre                 0
keyword

In [8]:
# Drop the 'Unnamed: 0' column
df = df.drop(columns=['Unnamed: 0'])

In [89]:
# Fill missing values for year_of_release
df['year_of_release'] = df['year_of_release'].fillna("Unknown")

# Fill missing values for actors with "Unknown"
df['actors'] = df['actors'].fillna("Unknown")

In [90]:
# Check for missing values
print(df.isnull().sum())

imdb-id             0
movie_name          0
year_of_release     0
runtime             0
IMDB_rating         0
no_of_votes         0
plot_description    0
director            0
actors              0
genre               0
dtype: int64


In [91]:
import numpy as np

# Replace empty strings or spaces with NaN
df['IMDB_rating'] = df['IMDB_rating'].replace(' ', np.nan)

# Convert IMDB_rating to float, handling missing values
df['IMDB_rating'] = pd.to_numeric(df['IMDB_rating'], errors='coerce')

# Fill NaN ratings with a placeholder value, e.g., the average rating
df['IMDB_rating'] = df['IMDB_rating'].fillna(df['IMDB_rating'].mean())

In [93]:
# Replace empty strings or invalid values with NaN in no_of_votes
df['no_of_votes'] = df['no_of_votes'].replace(' ', np.nan)

# Convert no_of_votes to integer, handling missing values
df['no_of_votes'] = pd.to_numeric(df['no_of_votes'].str.replace(",", ""), errors='coerce')

# Fill NaN values in no_of_votes with a default value, e.g., 0
df['no_of_votes'] = df['no_of_votes'].fillna(0).astype(int)

In [ ]:
# Verify cleaned columns
print(df['IMDB_rating'].head())
print(df['no_of_votes'].head())

# Check data types again
print(df.info())

0    8.3
1    8.4
2    8.3
3    8.3
4    8.1
Name: IMDB_rating, dtype: float64
0    548031
1    387020
2    188938
3    183452
4    180108
Name: no_of_votes, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   imdb-id           9999 non-null   object 
 1   movie_name        9999 non-null   object 
 2   year_of_release   9999 non-null   object 
 3   runtime           9999 non-null   object 
 4   IMDB_rating       9999 non-null   float64
 5   no_of_votes       9999 non-null   int64  
 6   plot_description  9999 non-null   object 
 7   director          9999 non-null   object 
 8   actors            9999 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 703.2+ KB
None


In [95]:
df['director'] = df['director'].apply(lambda x: x.split(", ") if x != "Unknown" else [])

In [96]:
df['genre'] = df['genre'].apply(lambda x: x.split(", ") if x != "Unknown Genre" else [])

In [99]:
# Check the first few rows of the processed columns
print(df['actors'].head())
print(df['director'].head())

0    [Vivek Agnihotri, Mithun Chakraborty, Anupam K...
1    [Rajkumar Hirani, Aamir Khan, Madhavan, Mona S...
2    [Aamir Khan, Amole Gupte, Darsheel Safary, Aam...
3    [Nitesh Tiwari, Aamir Khan, Sakshi Tanwar, Fat...
4    [Rajkumar Hirani, Aamir Khan, Anushka Sharma, ...
Name: actors, dtype: object
0    [Vivek Agnihotri]
1    [Rajkumar Hirani]
2         [Aamir Khan]
3      [Nitesh Tiwari]
4    [Rajkumar Hirani]
Name: director, dtype: object


In [100]:
print(df[['actors', 'director']].head())

                                              actors           director
0  [Vivek Agnihotri, Mithun Chakraborty, Anupam K...  [Vivek Agnihotri]
1  [Rajkumar Hirani, Aamir Khan, Madhavan, Mona S...  [Rajkumar Hirani]
2  [Aamir Khan, Amole Gupte, Darsheel Safary, Aam...       [Aamir Khan]
3  [Nitesh Tiwari, Aamir Khan, Sakshi Tanwar, Fat...    [Nitesh Tiwari]
4  [Rajkumar Hirani, Aamir Khan, Anushka Sharma, ...  [Rajkumar Hirani]


### Adding keywords feature to dataset according to plot description.

In [5]:
!pip install yake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 6.0 MB/s eta 0:00:00


In [5]:
import yake

print("Extracting keywords from plot descriptions...")
kw_extractor = yake.KeywordExtractor(top=5, stopwords=None)

def extract_keywords(text):
    try:
        keywords = kw_extractor.extract_keywords(text)
        return [kw for kw, score in keywords]
    except:
        return []

Extracting keywords from plot descriptions...


In [6]:
df['keywords'] = df['plot_description'].apply(extract_keywords)
print("Keywords extracted successfully.")

Keywords extracted successfully.


In [9]:
df.to_csv('final_dataset.csv', index=False)

# Getting Data ready to export to Neo4j Aura

In [104]:
# Select movie-related columns and save them
movies = df[['imdb-id', 'movie_name', 'year_of_release', 'runtime', 'IMDB_rating', 'no_of_votes', 'plot_description']].drop_duplicates()
movies.rename(columns={
    'imdb-id': 'imdb_id',
    'movie_name': 'title',
    'year_of_release': 'year',
    'runtime': 'runtime',
    'IMDB_rating': 'rating',
    'no_of_votes': 'votes',
    'plot_description': 'plot'
}, inplace=True)

# Save to CSV
movies.to_csv('movies.csv', index=False)
print("movies.csv created successfully.")

movies.csv created successfully.


In [106]:
# Actors DataFrame
unique_actors = list(set(actor.strip() for sublist in df['actors'] for actor in sublist if actor.strip()))
actors = pd.DataFrame({'name': unique_actors})
actors.to_csv('actors.csv', index=False)
print("actors.csv created successfully.")

# Directors DataFrame
unique_directors = list(set(director.strip() for sublist in df['director'] for director in sublist if director.strip()))
directors = pd.DataFrame({'name': unique_directors})
directors.to_csv('directors.csv', index=False)
print("directors.csv created successfully.")

# Genres DataFrame
movie_genre = df[['imdb-id', 'genre']].explode('genre').dropna()
movie_genre.rename(columns={'imdb-id': 'imdb_id'}, inplace=True)
unique_genres = movie_genre['genre'].unique()
genres = pd.DataFrame({'name': unique_genres})
genres.to_csv('genres.csv', index=False)
print("genres.csv created successfully.")

# Keywords DataFrame
movie_keyword = df[['imdb-id', 'keywords']].explode('keywords').dropna()
movie_keyword.rename(columns={'imdb-id': 'imdb_id', 'keywords': 'keyword'}, inplace=True)
unique_keywords = movie_keyword['keyword'].unique()
keywords = pd.DataFrame({'name': unique_keywords})
keywords.to_csv('keywords.csv', index=False)
print("keywords.csv created successfully.")

# Step 4: Prepare DataFrames for Relationships

# Movie-Actor Relationships
movie_actor = df[['imdb-id', 'actors']].explode('actors').dropna()
movie_actor['actors'] = movie_actor['actors'].str.strip()
movie_actor.rename(columns={'imdb-id': 'imdb_id', 'actors': 'actor_name'}, inplace=True)
movie_actor.to_csv('movie_actor.csv', index=False)
print("movie_actor.csv created successfully.")

# Movie-Director Relationships
movie_director = df[['imdb-id', 'director']].explode('director').dropna()
movie_director['director'] = movie_director['director'].str.strip()
movie_director.rename(columns={'imdb-id': 'imdb_id', 'director': 'director_name'}, inplace=True)
movie_director.to_csv('movie_director.csv', index=False)
print("movie_director.csv created successfully.")

# Movie-Genre Relationships (already created above)
movie_genre.to_csv('movie_genre.csv', index=False)
print("movie_genre.csv created successfully.")

# Movie-Keyword Relationships (already created above)
movie_keyword.to_csv('movie_keyword.csv', index=False)
print("movie_keyword.csv created successfully.")


actors.csv created successfully.
directors.csv created successfully.
genres.csv created successfully.
keywords.csv created successfully.
movie_actor.csv created successfully.
movie_director.csv created successfully.
movie_genre.csv created successfully.
movie_keyword.csv created successfully.
